# 根据不同年龄安装某个app的数量的标准差来确定app的重要程度
## 选出N个最重要的app作为特征
### 保存重要app文件：importance_app_inActivate.csv文件

In [1]:
import numpy as np
import pandas as pd

In [2]:
age_train = pd.read_csv('../data/age_train.csv',header=None)
user_app_actived = pd.read_csv('../data/user_app_actived.csv',header=None)

In [3]:
print(age_train.shape)
print(age_train.head())
print(user_app_actived.shape)
print(user_app_actived.head())

(2010000, 2)
         0  1
0  1000001  4
1  1000011  3
2  1000015  5
3  1000019  3
4  1000023  2
(2512500, 2)
         0                                                  1
0  1000008  a00105157#a00109386#a00134746#a0013952#a001451...
1  1000010  a00102782#a00107077#a00109386#a00134746#a00136...
2  1000011  a00158535#a00163116#a00170432#a00187480#a00224...
3  1000013  a00109386#a0011601#a0012768#a00136287#a0014516...
4  1000014  a00109386#a0011894#a0013059#a00134840#a0013578...


In [4]:
appNum,i = {}, 0 
for user_info in user_app_actived.values:
    appIdList = user_info[1].split('#')
    if appIdList[0]!='\\N':
        for app in appIdList:
            if app not in appNum.keys():
                appNum[app] = 0
            else:
                appNum[app] += 1

In [5]:
appNumList = np.array(list(appNum.values()))
boolNumList = appNumList>500
appNumMore500 = np.array(list(appNum.keys()))[boolNumList]

In [6]:
len(appNumMore500)

6936

In [7]:
user_app_actived = pd.merge(age_train,user_app_actived,how='inner',on=0)
print(user_app_actived.shape)
print(user_app_actived.head())

(2010000, 3)
         0  1_x                                                1_y
0  1000001    4  a00140327#a00170298#a00184278#a00187480#a00239...
1  1000011    3  a00158535#a00163116#a00170432#a00187480#a00224...
2  1000015    5  a00109386#a00170432#a0021880#a00244790#a002475...
3  1000019    3                                a00157060#a00274701
4  1000023    2  a00170298#a00203358#a00275200#a00278905#a00289...


### 所有用户的安装激活app总数为9400
### 训练集用户的安装激活app总数为9399

In [8]:
import re
col_index = {} #{appId:column}
i=0
for user_info in user_app_actived.values:
    appIdList = re.split('#',user_info[2])
    if appIdList[0]!='\\N':
        for app in appIdList:
            if app not in col_index.keys():
                col_index[app]=i
                i+=1

In [9]:
len(col_index.keys())

9399

In [10]:
age_appId_number_matrix = np.zeros((6,len(col_index.keys())))

In [11]:
for user_info in user_app_actived.values:
    appIdList = re.split('#',user_info[2])
    if appIdList[0]!='\\N':
        for app in appIdList:
            col = col_index[app]
            row = user_info[1]-1
            age_appId_number_matrix[row][col] += 1

In [12]:
ageNumber = age_train[1].value_counts()
ageNumber

3    600000
4    500000
2    400000
5    300000
6    150000
1     60000
Name: 1, dtype: int64

In [13]:
age_appId_number_matrix[0] = age_appId_number_matrix[0]/ageNumber[1]
age_appId_number_matrix[1] = age_appId_number_matrix[1]/ageNumber[2]
age_appId_number_matrix[2] = age_appId_number_matrix[2]/ageNumber[3]
age_appId_number_matrix[3] = age_appId_number_matrix[3]/ageNumber[4]
age_appId_number_matrix[4] = age_appId_number_matrix[4]/ageNumber[5]
age_appId_number_matrix[5] = age_appId_number_matrix[5]/ageNumber[6]
age_appId_number_matrix

array([[4.29666667e-02, 9.76666667e-03, 7.18333333e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.82225000e-02, 7.94250000e-03, 4.31500000e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.54800000e-02, 7.16500000e-03, 3.41333333e-03, ...,
        0.00000000e+00, 1.66666667e-06, 1.66666667e-06],
       [1.89440000e-02, 6.44600000e-03, 5.15400000e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.45200000e-02, 6.20333333e-03, 7.93666667e-03, ...,
        3.33333333e-06, 0.00000000e+00, 0.00000000e+00],
       [6.98000000e-03, 6.40666667e-03, 9.78000000e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [14]:
age_appId_number_matrix = pd.DataFrame(age_appId_number_matrix,columns=col_index.keys())
age_appId_number_matrix.describe()

,a00140327,a00170298,a00184278,a00187480,a00239194,a00244790,a00265462,a00276196,a00282018,a00289511,...,a00289133,a00294860,a00229848,a00270235,a00117775,a00128441,a00232931,a00223894,a00209087,a00467328
count,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,...,6.000000e+00,6.000000,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00
mean,0.034519,0.007322,0.006297,0.271009,0.045561,0.606670,0.027221,0.183261,0.015616,0.345057,...,3.333333e-07,0.000002,6.111111e-07,5.555556e-07,1.250000e-06,2.777778e-07,2.777778e-07,5.555556e-07,2.777778e-07,2.777778e-07
std,0.029520,0.001360,0.002415,0.058059,0.033852,0.087208,0.019933,0.056713,0.004528,0.127006,...,8.164966e-07,0.000002,9.525793e-07,1.360828e-06,1.467235e-06,6.804138e-07,6.804138e-07,1.360828e-06,6.804138e-07,6.804138e-07
min,0.006980,0.006203,0.003413,0.178733,0.002067,0.444117,0.002567,0.127300,0.010388,0.213150,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.015626,0.006417,0.004525,0.241559,0.018228,0.591003,0.009641,0.138662,0.012536,0.248484,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.027212,0.006806,0.006169,0.292036,0.053453,0.633138,0.031896,0.166764,0.015543,0.324151,...,0.000000e+00,0.000002,0.000000e+00,0.000000e+00,8.333333e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.041095,0.007748,0.007748,0.297784,0.069074,0.662693,0.043633,0.231272,0.017062,0.415498,...,0.000000e+00,0.000003,1.250000e-06,0.000000e+00,2.291667e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,0.088222,0.009767,0.009780,0.338815,0.083624,0.680125,0.047200,0.256667,0.023117,0.539680,...,2.000000e-06,0.000005,2.000000e-06,3.333333e-06,3.333333e-06,1.666667e-06,1.666667e-06,3.333333e-06,1.666667e-06,1.666667e-06


In [15]:
app_stds = age_appId_number_matrix.std()
app_stds.sort_values(ascending=False,inplace=True)

In [16]:
age_appId_number_matrix = age_appId_number_matrix.reindex(columns=app_stds.keys())

In [17]:
age_appId_number_matrix.describe()

,a00224427,a00292780,a00292472,a00481059,a00278905,a00290015,a00290027,a00289511,a00289826,a00275200,...,a00171281,a00128927,a00229848,a00289133,a00209087,a00140477,a00128441,a00232931,a00254405,a00467328
count,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,...,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00
mean,0.266058,0.231940,0.213439,0.172900,0.447292,0.480747,0.361929,0.345057,0.766036,0.277573,...,7.500000e-07,1.027778e-06,6.111111e-07,3.333333e-07,2.777778e-07,2.777778e-07,2.777778e-07,2.777778e-07,2.777778e-07,2.777778e-07
std,0.233128,0.229186,0.209644,0.159648,0.136621,0.134584,0.127727,0.127006,0.123266,0.118294,...,1.172604e-06,1.156703e-06,9.525793e-07,8.164966e-07,6.804138e-07,6.804138e-07,6.804138e-07,6.804138e-07,6.804138e-07,6.804138e-07
min,0.054720,0.016160,0.018067,0.025040,0.248333,0.319273,0.259353,0.213150,0.580040,0.152007,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.094457,0.050983,0.053606,0.053405,0.357908,0.372073,0.282902,0.248484,0.709381,0.199341,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.168868,0.170615,0.148165,0.111001,0.479465,0.482260,0.298842,0.324151,0.762080,0.243784,...,0.000000e+00,8.333333e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.462131,0.367525,0.339015,0.310763,0.529467,0.579273,0.417824,0.415498,0.846644,0.342535,...,1.500000e-06,1.916667e-06,1.250000e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,0.574350,0.589983,0.541517,0.378800,0.612967,0.653733,0.579467,0.539680,0.925367,0.465883,...,2.500000e-06,2.500000e-06,2.000000e-06,2.000000e-06,1.666667e-06,1.666667e-06,1.666667e-06,1.666667e-06,1.666667e-06,1.666667e-06


In [18]:
app_keys = age_appId_number_matrix.keys()
app_keys

Index(['a00224427', 'a00292780', 'a00292472', 'a00481059', 'a00278905',
       'a00290015', 'a00290027', 'a00289511', 'a00289826', 'a00275200',
       ...
       'a00171281', 'a00128927', 'a00229848', 'a00289133', 'a00209087',
       'a00140477', 'a00128441', 'a00232931', 'a00254405', 'a00467328'],
      dtype='object', length=9399)

In [19]:
appMore500AndStd = []
for app in app_keys:
    if app in appNumMore500:
        appMore500AndStd.append(app)

In [20]:
importance_app = appMore500AndStd[:4000]

In [21]:
importance_app = pd.DataFrame(importance_app)
importance_app.to_csv('../processed/importance_app_inActivate.csv',header=None,index=False)
importance_app.shape

(4000, 1)